<a href="https://colab.research.google.com/github/Mohitnayak/haystack/blob/main/haystack_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Jun 21 16:36:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.9 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-ylxqsnpq/farm-haystack_ae59c69b1e6e4087bb811a79937cb8ef
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-ylxqsnpq/farm-haystack_ae59c69b1e6e4087bb811a79937cb8ef
  Resolved https://github.com/deepset-ai/haystack.git to commit 4a63707f1a177123c13929eb316d3ecaa7fd6c5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 9.

In [3]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

/usr/local/lib/python3.7/dist-packages/huggingface_hub/snapshot_download.py:11: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  FutureWarning,
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [6]:
# # Recommended: Start Elasticsearch using Docker via the Haystack utility function
# from haystack.utils import launch_es

# launch_es()


# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [7]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [9]:
doc_dir = "data/tutorial1"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip to `data/tutorial1`


True

In [10]:
# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

INFO - haystack.utils.preprocessing -  Converting data/tutorial1/120_And_Now_His_Watch_Is_Ended.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/133_Game_of_Thrones__Season_5__soundtrack_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/99_A_Song_of_Ass_and_Fire.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/119_Walk_of_Punishment.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/481_Sandor_Clegane.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/154_Margaery_Tyrell.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/460_Battle_of_the_Bastards.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/394_Game_of_Thrones__2014_video_game_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/395_Game_of_Thrones__season_5_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/67_Unbowed__Unbent__Unbroken.txt
INFO - haystack.utils.p

In [12]:
docs[:3]

[<Document: {'content': '"\'\'\'And Now His Watch Is Ended\'\'\'" is the fourth episode of the third season of HBO\'s fantasy television series \'\'Game of Thrones\'\', and the 24th episode of the series. It was written by showrunners and executive producers David Benioff and D. B. Weiss and directed by Alex Graves, his directorial debut for the series.\nThe episode\'s title comes from a chant made by the Night\'s Watch at the funeral of a fallen brother while at Craster\'s Keep.', 'content_type': 'text', 'score': None, 'meta': {'name': '120_And_Now_His_Watch_Is_Ended.txt'}, 'embedding': None, 'id': '7cf938c349c53f9af9ac80fb219af781'}>,
 <Document: {'content': "\n===In King's Landing===\nVarys tells Tyrion of being made a eunuch by a sorcerer in Myr, before revealing that he has captured the sorcerer and plans to take revenge on him. Ros informs Varys that Baelish may take Sansa when he leaves to wed Lysa Arryn, and Varys warns Olenna that Baelish may seek to control Sansa.\nDiscussing

In [13]:
document_store.write_documents(docs)

In [15]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [16]:
retriever

In [17]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [18]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [23]:
# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="Who is the father of Tyrion?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)



Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 45.35 Batches/s]


In [24]:
print_answers(prediction, details="minimum")


Query: Who is the father of Tyrion?
Answers:
[   {   'answer': 'Tywin',
        'context': 'Stark marrying two of his children.\n'
                   'Tyrion Lannister suspects his father Tywin, who decides '
                   'Tyrion and his barbarians will fight in the vanguard, '
                   'want'},
    {   'answer': 'Tywin',
        'context': 'ellsword Bronn, who is victorious.  The two meet up with '
                   "Tyrion's father Tywin, whose forces are fighting Robb "
                   "Stark's army as retaliation for his cap"},
    {   'answer': 'Tywin',
        'context': "my camp. Finally impressed with Tyrion's political "
                   'instincts, his father Tywin appoints Tyrion acting Hand of '
                   'the King in an attempt to control Joffre'},
    {   'answer': 'Tywin',
        'context': ' Catelyn Stark and her sister Lysa Arryn, and then is sent '
                   'by his father Tywin to impose order on the capital of '
            